In [1]:
import os, re
import tensorflow as tf
import pandas as pd
import numpy as np


import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.optimizers as Optim
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [3]:
def build_model(transformer, max_len = 512):
    input_word_ids = L.Input(shape = (max_len,), dtype = tf.int32, name = 'input_word_ids')
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = L.Dense(1, activation = 'sigmoid')(cls_token)
    
    model = Model(inputs = input_word_ids, outputs = out)
    model.compile(Optim.Adam(lr = 1e-5), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [4]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()


Running on TPU  grpc://10.0.0.2:8470


In [14]:
AUTO = tf.data.experimental.AUTOTUNE

# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# Configuration
TRAIN_EPOCHS = 5
VALID_EPOCHS = 5
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192
MODEL = 'jplu/tf-xlm-roberta-large'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.save_pretrained('.')

('./sentencepiece.bpe.model',
 './special_tokens_map.json',
 './added_tokens.json')

In [7]:
train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
train2.toxic = train2.toxic.round().astype(int)

valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [8]:
train = pd.concat([
    train1[['comment_text', 'toxic']],
    train2[['comment_text', 'toxic']].query('toxic==1'),
    train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=0)
])

In [9]:
train.shape

(435775, 2)

In [16]:
x_train = regular_encode(train.comment_text.values, tokenizer, maxlen = MAX_LEN)
x_valid = regular_encode(valid.comment_text.values, tokenizer, maxlen = MAX_LEN)
"""x_test = regular_encode(test.content.values, tokenizer, maxlen = MAX_LEN)"""

y_train = train.toxic.values
y_valid = valid.toxic.values

In [17]:
train_dataset = (tf.data.Dataset
                .from_tensor_slices((x_train, y_train))
                .repeat()
                .shuffle(2048)
                .batch(BATCH_SIZE)
                .prefetch(AUTO)
                )

valid_dataset = (tf.data.Dataset
                .from_tensor_slices((x_valid, y_valid))
                .cache()
                .batch(BATCH_SIZE)
                .prefetch(AUTO)
                )

"""test_dataset = (tf.data.Dataset
                .from_tensor_slices(x_test)
                .batch(BATCH_SIZE)
                )"""

'test_dataset = (tf.data.Dataset\n                .from_tensor_slices(x_test)\n                .batch(BATCH_SIZE)\n                )'

In [12]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 512)]             0         
_________________________________________________________________
tf_roberta_model (TFRobertaM ((None, 512, 1024), (None 559890432 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_________________________________________________________________
CPU times: user 2min 7s, sys: 43.2 s, total: 2min 50s
Wall time: 2min 49s


In [13]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=5
)

Epoch 1/5


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:431: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)


ResourceExhaustedError: {{function_node __inference_train_function_142559}} Compilation failure: Ran out of memory in memory space hbm. Used 16.71G of 15.48G hbm. Exceeded hbm capacity by 1.23G.

Total hbm usage >= 17.23G:
    reserved        529.00M 
    program          16.71G 
    arguments       unknown size 

Output size unknown.

Program hbm requirement 16.71G:
    reserved           4.0K
    global           356.0K
    scoped           14.17M
    HLO temp         16.70G (99.9% utilization: Unpadded (16.67G) Padded (16.69G), 0.1% fragmentation (11.84M))

  Largest program allocations in hbm:

  1. Size: 256.00M
     Operator: op_type="BatchMatMulV2" op_name="model/tf_roberta_model/roberta/encoder/layer_._17/attention/self/MatMul"
     Shape: f32[16,16,512,512]{2,3,1,0:T(8,128)}
     Unpadded size: 256.00M
     XLA label: %fusion.3978.remat3 = (f32[16,16,512]{2,1,0:T(8,128)}, f32[16,16,512]{2,1,0:T(8,128)}, f32[16,16,512,512]{2,3,1,0:T(8,128)}) fusion(s32[]{:T(256)} %get-tuple-ele

In [ ]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=5
)

In [ ]:
copy = build_model(transformer_layer, max_len - MAX_LEN)
copy.set_weights(model.get_weights())
copy.save('UnToxiK.h5')